A _multiplex stream graph_ $M = (T,T_M,V,W_M,E_M,{\cal L})$

- $T$ is an interval
- $T_M$ is a set of intervals
- $V$ is a set of nodes
- $W_M$ is a set of (t,u,\alpha_1,...,\alpha_d)
- $E_M$ is a set of (t,u,\alpha_1,...,\alpha_d, v,\beta_1,...,\beta_d)
- ${\cal L}$ is a set of aspects, containing elementary layers.


We begin by constructing **"interval"** object and a few methods.

We the contruct the lists of intervals which respects the following contraints : 
- the intervals are separated,
- the intervals are sorted on crescent time


In [1]:
class Interval :
    def __init__(self,t1,t2):
        self.t1 = t1
        self.t2 = t2
    
    def isIn(self,x):
        if x >= (self.t1) and x <= (self.t2) :
            return True
        else :
            return False
    
    def length(self):
        return (self.t2-self.t1)
    def begining(self):
        return(self.t1)
    def end(self):
        return(self.t2)
    def setBegining(self,b):
        self.t1=b
    def setEnd(self,e):
        self.t2=e
    def intersect(self,int2):
        if self.isIn(int2.begining()) or self.isIn(int2.end()) :
            return(True)
        elif int2.isIn(self.begining()) or int2.isIn(self.end()):
            return(True)
        else :
            return(False)
    def intersection(self,int2):
        if self.intersect(int2):
            return Interval(max(self.t1,int2.begining()),min(self.t2,int2.end()))
        else :
            return Interval(0,0)
    def union(self,int2):#a n'appliquer que dans le cas ou deux intervalles sont connexes
        return(Interval(min(self.begining(),int2.begining()),max(self.end(),int2.end())))
    def intervalToString(self):
        return("["+str(self.t1)+","+str(self.t2)+"]")
    def contains(self,int2): #tests if contains int2
        if (self.t1 <= int2.begining()) & (self.t2 >= int2.end()):
            return(True)
        else :
            return(False)

#functions on lists of intervals.
        
def sortIntervals(listOfIntervals):
    listOfIntervals.sort(key=lambda interval: interval.begining())
    return(listOfIntervals)

def printListIntervals(listOfIntervals):
    print("list of intervals")
    for i in listOfIntervals:
        print(i.intervalToString())

def condensateIntervals(listOfIntervals):
    listOfIntervals=sortIntervals(listOfIntervals)
    i=0;
    while i<(len(listOfIntervals)-1):
        if listOfIntervals[i].end()>listOfIntervals[i+1].begining():
            inte=listOfIntervals.pop(i+1)
            listOfIntervals[i].setEnd(max(inte.end(),listOfIntervals[i].end()))
        else :
            i=i+1
    return(listOfIntervals)

def addInterval(loi,interval):#loi : list of intervals
    #to do : chercher la place de l'intervalle par dichotomie (sur les débuts des intervalles) puis condenser à partir de l'indice de l'intervalle d'avant.
    if interval.begining()<loi[0].begining():
        loi.insert(0,interval)
        condensateIntervals(loi)
        return(loi)
    elif interval.begining()>loi[-1].end():
        loi.append(interval)
        return(loi)
    else :
        i=0
        j=len(loi)-1
        k=0
        while i<j-1:
            print("i:",i,",j:",j)
            k=int(i+j)//2
            if loi[k].begining()>interval.begining():
                j=k
            elif loi[k].begining()<interval.begining():
                i=k
            else :
                i=k
                j=k
        loi.insert(k+1,interval)
        l=k;
        while l<(len(loi)-1):
            if loi[l].end()>loi[l+1].begining():
                inte=loi.pop(l+1)
                loi[l].setEnd(max(inte.end(),loi[l].end()))
            else :
                l=l+1
        return(loi)
    
#test functions 

int1 = Interval(0,10)
int2 = Interval(11,15)
int3 = Interval(9,13)
int5= Interval(5,8)
int6=Interval(20,25)
int4 = int3.intersection(int1)
int1.contains(int5)

listOf=[int1,int2,int3,int4,int5,int6]
l=sortIntervals(listOf)

printListIntervals(listOf)
condensateIntervals(listOf)
printListIntervals(listOf)
addInterval(listOf,Interval(18.3,19))
printListIntervals(listOf)

list of intervals
[0,10]
[5,8]
[9,13]
[9,10]
[11,15]
[20,25]
list of intervals
[0,15]
[20,25]
list of intervals
[0,15]
[18.3,19]
[20,25]


We contruct then the class **Aspects** which has a `name`and a list of `elementary layers`.

*Remarque : à voir si on leur attribue des temps d'existence ou non. Pour l'instant non à priori, ça sera fait directement au niveau des layers*

In [2]:
class Aspect : 
    def __init__(self,name,elemLayer):
        self.name=name
        self.elemLayer = elemLayer
        
    def nameAspect(self):
        return(self.name)
    def listElemLayer(self):
        return(self.elemLayer)
    def printAspect(self):
        print(self.name," : ", self.elemLayer)
    def aspectToString(self):
        stri=""
        stri=stri +str(self.name) +" : " + str(self.elemLayer)
        return(stri)
    
    
uni = Aspect("Université",["Polytechnique","Todai","Paris6"] )
dept = Aspect("departement",["mathematiques","biologie","informatique","mecanique"])
poste = Aspect("Poste",["stagiaire","these","post-doc","chercheur"])
print(uni.nameAspect())
print(uni.listElemLayer())
dept.printAspect()
print(dept.aspectToString())

Université
['Polytechnique', 'Todai', 'Paris6']
departement  :  ['mathematiques', 'biologie', 'informatique', 'mecanique']
departement : ['mathematiques', 'biologie', 'informatique', 'mecanique']


La classe **LayerT** sert à stocker une couche ainsi que son temps d'existence qui doit être un intervalle.

In [3]:
class LayerT :
    def __init__(self,layer,interval):#layer is a list of elementary layers from Layers
        self.layer=layer
        self.interval=interval
    def setInterval(self,interval2):
        self.interval=interval2
    def getInterval(self):
        return self.interval
    def getlayer(self):
        return(self.layer)
    def printLayerT(self):
        print("layer: ",self.layer, "interval", self.interval.intervalToString())
    def layerTToString(self):
        return("layer: ",self.layer, "interval", self.interval.intervalToString())

We create then the class **Layers** containing `aspects` (list of **Aspect**).

The recursive method `buildLayers` is able to do all the combinations of elementary layers and to construct all the layers possible. It may be usefull for a graphic representation in the future *(idea : just to separate the different layers with some spaces ...)*

In [4]:
class Layers :
    def __init__(self,aspects):
        self.aspects=aspects
    def addAspect(self, aspect):
        self.aspects.append(aspect)
    def printLayers(self):
        for i in self.aspects :
            i.printAspect()
    def buildLayer(self):
        aspect2 = self.aspects.copy()
        def buildLayerRec(listOfAspects):
            if len(listOfAspects)==0:
                return([[]])
            else :
                elemLayers=(listOfAspects.pop()).listElemLayer()
                layers = buildLayerRec(listOfAspects)
                lengthLayersList = len(layers)
                for j in range(lengthLayersList) :
                    l=layers.pop(0)
                    for j in elemLayers :
                        l.append(j)
                        layers.append(l.copy())
                        l.pop()
                return(layers)
        return(buildLayerRec(aspect2))
    
    def buildLayerT(self,interval):
        layers = self.buildLayer()
        layersT=[]
        for l in layers : 
            layersT.append(LayerT(l,interval))
        return(layersT)

def printListOfLayerT(liste):
    for l in list : 
        l.printLayerT()
    
layers = Layers([uni,dept])
layers.addAspect(poste)
print("***************layers : *****************")
layers.printLayers()
print("*************end***************")
listeLayers = layers.buildLayer()
#print(listeLayers)
layerT=layers.buildLayerT(Interval(0,10))
#for l in layerT :
 #   l.printLayerT()

***************layers : *****************
Université  :  ['Polytechnique', 'Todai', 'Paris6']
departement  :  ['mathematiques', 'biologie', 'informatique', 'mecanique']
Poste  :  ['stagiaire', 'these', 'post-doc', 'chercheur']
*************end***************


We need then to determine when the nodes appear in each layer. We create **NodeLayerT** in that purpose.

We add some contraints on `LT`, the list of times at which the node appears :
- the intervals are list of intervals :
    - the intervals are separated,
    - the intervals are sorted on crescent 
- the intervals are included in the existence interval of layerT

In [16]:
class NodeLayerT :
    def __init__(self,node,layerT,LT):
        self.node=node
        self.layerT=layerT
        self.LT=[]#lt is a list of intervals
        for i in LT : #check if the existence intervals of the node-layer is included in the existence interval of the layer.
            if (layerT.getInterval()).contains(i):
                self.LT.append(i)
            else :
                print("error : the node-layer cannont exist at",i.intervalToString(),"interval removed")
        condensateIntervals(self.LT)
        
    
    def presentAtt(self,t):
        for T in self.LT :
            if T.isIn(t):
                return(True)
        return(False)
    def issamenodelayer(self,nodelayert): #checks if two node-layers are the same.
        if self.node == nodelayert.node :
            if self.layer == nodelayert.layer:
                return(True)
            return(False)
        return(False)
    def addTime(self,interval):
        addInterval(self.LT,interval)
    def givetimes(self):
        return(self.LT)
    def printNodeLayerT(self):
        strLT=""
        for i in range(len(self.LT)):
            strLT+=self.LT[i].intervalToString()
        print("nodeLayerT","node:",self.node,"of",self.layerT.layerTToString(),"exists at LT:", strLT )

    def toString(self):
        strLT=""
        for i in range(len(self.LT)):
            strLT+=self.LT[i].intervalToString()
        return("nodeLayerT","node:",self.node,"of",self.layerT.layerTToString(),"exists at LT:", strLT )
        
layT = layerT.pop()
print("coucou")
layT.printLayerT()
print("bye")
n=NodeLayerT("chercheurA",layT,[Interval(0,11),Interval(3,4),Interval(1,2),Interval(6,9)])
n.printNodeLayerT()
#n.addtime([Interval(5,6),Interval(4,7)])
n.printNodeLayerT()

coucou
layer:  ['Paris6', 'mecanique', 'these'] interval [0,10]
bye
error : the node-layer cannont exist at [0,11] interval removed
nodeLayerT node: chercheurA of ('layer: ', ['Paris6', 'mecanique', 'these'], 'interval', '[0,10]') exists at LT: [1,2][3,4][6,9]
nodeLayerT node: chercheurA of ('layer: ', ['Paris6', 'mecanique', 'these'], 'interval', '[0,10]') exists at LT: [1,2][3,4][6,9]


**Link** contains the two node layers and the intervals of existence

In [22]:
class Link:
    def __init__(self,nodeLayerT1,nodeLayerT2,LT):
        self.nodeLayerT1=nodeLayerT1
        self.nodeLayerT2=nodeLayerT2
        self.LT=condensateIntervals(LT)
    
    def addTime(self):
        #todo
        return()
        
    def removeTime(self):
        #todo
        return()
    def printLink(self):
        strLT=""
        for i in range(len(self.LT)):
            strLT+=self.LT[i].intervalToString()
        print(self.nodeLayerT1.toString(),"<-->",self.nodeLayerT1.toString(),strLT)

**Multistream** is our final object.

In [23]:
class MultiStream :
    
    def __init__(self,T,V,L):
        self.T=T #an interval for the existence of the graph
        self.V=V #the nodes
        self.TM = L.buildLayerT(T)
        self.WM=[] #node layers are built from L
        self.EM=[] #links are empty at the begining
        self.L=L #layers structure
    def interval(self):
        return(self.T)
    def addNodeLayerT(self, nodelayert): #quand on rajoute un t-node-layer, soit il existe déjà à d'autres temps donc on fusionne les intervalles d'existence, soit on le rajoute avec ses intervalles
        if len(self.WM)==0:
            self.WM.append(nodelayert)
        else:
            for inlt in range(len(self.WM)) : #self.WM[inlt] is a node layer
                if (self.WM[inlt]).issamenodelayer():
                    for intervalToAdd in (nodelayert.givetimes()):
                        (self.WM[inlt]).addtime(intervalToAdd)
                else :
                    self.WM.append(nodelayert)
    def addLink(self,link):
        #to do : check intervals coherence
        self.EM.append(link)
    def printMS(self):
        print("T:",self.T.intervalToString(),",V : ",self.V )
        print("TM")
        for t in self.TM:
            t.printLayerT()
        print("WM")
        for i in self.WM:
            i.printNodeLayerT()
        print("EM")
        for em in self.EM:
            em.printLink()
        print("L")
        self.L.printLayers()
    
m=MultiStream(Interval(0,10),["p1","p2"],layers)
nlt=NodeLayerT("p1",layT,[Interval(6,7)])
nlt2=NodeLayerT("p2",layT,[Interval(6,9)])
m.addNodeLayerT(nlt)
m.addLink(Link(nlt,nlt2,[Interval(6,7)]))
m.printMS()

T: [0,10] ,V :  ['p1', 'p2']
TM
layer:  ['Polytechnique', 'mathematiques', 'stagiaire'] interval [0,10]
layer:  ['Polytechnique', 'mathematiques', 'these'] interval [0,10]
layer:  ['Polytechnique', 'mathematiques', 'post-doc'] interval [0,10]
layer:  ['Polytechnique', 'mathematiques', 'chercheur'] interval [0,10]
layer:  ['Polytechnique', 'biologie', 'stagiaire'] interval [0,10]
layer:  ['Polytechnique', 'biologie', 'these'] interval [0,10]
layer:  ['Polytechnique', 'biologie', 'post-doc'] interval [0,10]
layer:  ['Polytechnique', 'biologie', 'chercheur'] interval [0,10]
layer:  ['Polytechnique', 'informatique', 'stagiaire'] interval [0,10]
layer:  ['Polytechnique', 'informatique', 'these'] interval [0,10]
layer:  ['Polytechnique', 'informatique', 'post-doc'] interval [0,10]
layer:  ['Polytechnique', 'informatique', 'chercheur'] interval [0,10]
layer:  ['Polytechnique', 'mecanique', 'stagiaire'] interval [0,10]
layer:  ['Polytechnique', 'mecanique', 'these'] interval [0,10]
layer:  ['P